In [2]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from api_keys import DarkSkyKey

In [3]:
# read taxi rides file
file = "Resources/Taxi_Trips_Random_Sample.csv"
df = pd.read_csv(file, header=None)
df.columns = ["Trip ID", "Taxi ID","Trip Start Timestamp", "Trip End Timestamp","Trip Seconds",
             "Trip Miles", "Pickup Census Tract", "Dropoff Census Tract", "Pickup Community Area", 
             "Dropoff Community Area", "Fare", "Tips", "Tolls", "Extras", "Trip Total", "Payment Type", 
             "Company", "Pickup Centroid Latitude", "Pickup Centroid Longitude", "Pickup Centroid Location", 
             "Dropoff Centroid Latitude", "Dropoff Centroid Longitude", "Dropoff Centroid Location","Community Areas"]
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,11/16/2013 01:45:00 AM,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,09/28/2016 04:45:00 PM,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,12/10/2013 07:15:00 AM,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,02/27/2014 09:00:00 AM,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,10/31/2016 04:15:00 PM,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [4]:
# convert timestamp format for api calls
newdatestart_list = []

for date in df['Trip Start Timestamp']:
    date_obj = datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')
    new_date = date_obj.strftime('%Y-%m-%dT%H:%M:%S')
    newdatestart_list.append(new_date)

df['Trip Start Timestamp'] = newdatestart_list
df.head()


,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,2013-11-16T01:45:00,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,2016-09-28T04:45:00,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,2013-12-10T07:15:00,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,2014-02-27T09:00:00,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,2016-10-31T04:15:00,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [5]:
# Save config information
url = "https://api.darksky.net/forecast/"
exception_count = 0
n = 0

weather_df=pd.DataFrame()

# Build query URL and DataFrame
for i in range(1002, len(df)):
    #get info from taxi ride file
    lat = df['Pickup Centroid Latitude'][i]
    lng = df['Pickup Centroid Longitude'][i]
    time = df['Trip Start Timestamp'][i]

    #make API calls with this info
    try:
        query_url = url + DarkSkyKey + "/" + str(lat) +"," + str(lng) + "," + time 
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()['currently'] 
        weather_df.loc[i,'Trip ID'] = df['Trip ID'][i]
        weather_df.loc[i,'time'] = time
        weather_df.loc[i,'summary'] = weather_json['summary']
        weather_df.loc[i,'precipIntensity'] = weather_json['precipIntensity']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'temperature'] = weather_json['temperature']
        weather_df.loc[i,'apparentTemperature'] = weather_json['apparentTemperature']
        weather_df.loc[i,'windSpeed'] = weather_json['windSpeed']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'icon'] = weather_json['icon']
    
    except:
        print(f"Request no {i} failed, skipping to next line")
        
        if i-n == 1: exception_count += 1
        else: exception_count = 0
            
        n = i
        
        if exception_count >= 5: break
        else: continue

weather_df

Request no 1139 failed, skipping to next line
Request no 1221 failed, skipping to next line
Request no 1233 failed, skipping to next line
Request no 1400 failed, skipping to next line
Request no 1578 failed, skipping to next line
Request no 1835 failed, skipping to next line
Request no 1837 failed, skipping to next line
Request no 2009 failed, skipping to next line
Request no 2010 failed, skipping to next line
Request no 2011 failed, skipping to next line
Request no 2012 failed, skipping to next line
Request no 2013 failed, skipping to next line
Request no 2014 failed, skipping to next line
Request no 2015 failed, skipping to next line
Request no 2016 failed, skipping to next line
Request no 2017 failed, skipping to next line
Request no 2018 failed, skipping to next line
Request no 2019 failed, skipping to next line


,Trip ID,time,summary,precipIntensity,precipProbability,temperature,apparentTemperature,windSpeed,icon
1002,c0a815f44e25b1370f5246d3accf62802e94b514,2015-07-30T12:15:00,Clear,0.0000,0.00,84.84,84.84,4.52,clear-day
1003,b990b389212431212ffac69deb4a5540873583c2,2014-10-31T07:45:00,Overcast,0.0000,0.00,36.62,30.41,8.12,cloudy
1004,b6ffeb1d5a35efd11cef5d84ecfc6ec701699209,2013-05-25T02:00:00,Mostly Cloudy,0.0000,0.00,44.18,44.18,0.51,partly-cloudy-night
1005,bfb42d826dca0c5f0104823b3d240772f447f814,2013-08-14T10:00:00,Mostly Cloudy,0.0000,0.00,66.09,66.09,2.47,partly-cloudy-day
1006,c0110d814c9ae555638ab7a23acef1395d289fbf,2016-12-07T11:45:00,Partly Cloudy,0.0000,0.00,28.52,21.74,6.43,partly-cloudy-day
1007,c037199b2243fd6607ad2fe07b66830a5237a654,2015-07-24T08:15:00,Partly Cloudy,0.0000,0.00,77.79,78.28,0.27,partly-cloudy-day
1008,bd8616e748f801c0c0efcf056135b946fca9407d,2014-02-26T09:30:00,Clear,0.0000,0.00,5.61,-4.01,5.06,clear-day
1009,bdffba605fa91707616c3b7748a86d40543bc944,2016-08-23T10:45:00,Partly Cloudy,0.0000,0.00,76.85,77.08,5.19,partly-cloudy-day
1010,ba6acd6498a46d3049fee5acd4f0e5c27cd0ac92,2014-04-30T07:45:00,Overcast,0.0000,0.00,50.71,50.71,4.06,cloudy
1011,bca67203148b91bb5131b86456e2cc00a93c6edc,2013-08-19T12:15:00,Partly Cloudy,0.0000,0.00,81.28,82.01,1.68,partly-cloudy-day


In [6]:
weather_df['summary'].unique()


array(['Clear', 'Overcast', 'Mostly Cloudy', 'Partly Cloudy', 'Drizzle',
       'Humid and Partly Cloudy', 'Foggy', 'Light Rain', 'Humid',
       'Humid and Mostly Cloudy', 'Rain', 'Humid and Overcast',
       'Flurries', 'Light Snow'], dtype=object)

In [8]:
try: os.mkdir('Output')
except: pass

try:
    with open("Output/weather.csv", "a") as f:
        weather_df.to_csv(f, header=False)
except:
    weather_df.to_csv("Output/weather.csv")
